In [162]:
import random
import hashlib
import time
from collections import defaultdict

In [163]:
def h(s):
    return int(hashlib.sha256(str(s).encode('utf-8')).hexdigest(), 16) %100

def concat(a):
    return "".join([str(x) for x in a])

In [164]:
idi  = random.randint(1, 2**3)
sni = random.randint(1, 2**3)



In [165]:
k = random.randint(1, 2**3)
gateway_private = {"K" :k}
GW_private = defaultdict(dict)
SM_private = defaultdict(dict)

In [166]:
def GW_registration(params):
    idi = params[0]
    hsni = params[1]
    k = gateway_private["K"]
    xi = random.randint(1, 2**3)
    GW_private[idi]["xi"] = xi
    GW_private[idi]["hsni"] = hsni
    Ni = h(concat([idi, hsni])) ^ h(concat([k , xi]))
    return Ni, xi

In [167]:
def SM_init(uuid):
    idi  = random.randint(1, 2**3)
    sni = random.randint(1, 2**3)
    hsni = h(sni)
    SM_private[uuid] = {"idi" : idi, "sni" : sni, "hsni" : hsni}
    return (idi,  hsni)

In [168]:
uuid = 2
r = GW_registration(SM_init(uuid))  
SM_private[uuid]["Ni"] , SM_private[uuid]["xi"] = r[0], r[1]

In [169]:
SM_private

defaultdict(dict, {2: {'idi': 3, 'sni': 2, 'hsni': 61, 'Ni': 11, 'xi': 2}})

In [170]:

verifiction_params = GW_registration(r)


In [171]:
import time
import random


def SMI_verify(uuid):
    idi = SM_private[uuid]["idi"]
    hsni = SM_private[uuid]["hsni"]
    t1 = time.time()
    xi = SM_private[uuid]["xi"]
    ni = SM_private[uuid]["Ni"]

    print("idi:", idi)
    print("hsni:", hsni)
    print("t1:", t1)
    print("xi:", xi)
    print("ni:", ni)

    print(
        "h(concat([xi, t1])",
        h(concat([xi, t1])),
        "concat([idi, hsni]))",
        h(concat([idi, hsni])),
    )

    didi = h(concat([idi, hsni])) ^ h(concat([xi, t1]))

    ki = random.randint(1, 2**3)
    SM_private[uuid]["ki"] = ki

    a = ni ^ h(concat([idi, sni]))
    pki = h(concat([a, t1])) ^ ki
    bi = h(concat([ni, xi, t1, pki]))

    rparams = {}
    rparams["idi"] = idi
    rparams["didi"] = didi
    rparams["pki"] = pki
    rparams["bi"] = bi
    rparams["ti"] = t1

    print("didi:", didi)
    print("ki:", ki)
    print("pki:", pki)
    print("bi:", bi)
    print("rparams:", rparams)

    return rparams

In [172]:
SM_private[2]

{'idi': 3, 'sni': 2, 'hsni': 61, 'Ni': 11, 'xi': 2}

In [173]:
rparams = SMI_verify(2)
print(rparams)
print("gwidi" , GW_private)



idi: 3
hsni: 61
t1: 1710425031.5375154
xi: 2
ni: 11
h(concat([xi, t1]) 58 concat([idi, hsni])) 16
didi: 42
ki: 2
pki: 25
bi: 80
rparams: {'idi': 3, 'didi': 42, 'pki': 25, 'bi': 80, 'ti': 1710425031.5375154}
{'idi': 3, 'didi': 42, 'pki': 25, 'bi': 80, 'ti': 1710425031.5375154}
gwidi defaultdict(<class 'dict'>, {3: {'xi': 2, 'hsni': 61}, 11: {'xi': 3, 'hsni': 2}})


In [174]:
def GW_verify(rparams):
    t_ = time.time()
    t1 = rparams["ti"]
    dt = 10000
    if t_ - t1 > dt:
        print("time out")
    idi = rparams["idi"]
    xi = GW_private[rparams["idi"]]["xi"]
    hsni = GW_private[rparams["idi"]]["hsni"]
    lhs = h(concat([idi, hsni]))
    didi = rparams["didi"]
    ti  = rparams["ti"]
    rhs = didi ^ h(concat([xi, ti]))
    k = gateway_private["K"]
    pki = rparams["pki"]
    bi_ = h(concat([lhs ^ h(concat([k , xi])) , xi , ti , pki]))
    
    
    ki = h(h(concat([k, xi])))
    return True if bi_ == rparams["bi"] else False
    # print(bi_ == rparams["bi"])
# print(GW_verify(rparams))

In [175]:
GW_verify(rparams)

True